In [3]:
## from https://github.com/graykode/nlp-tutorial/tree/master/5-2.BERT
import math
import re
from random import *
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# sample IsNext and NotNext to be same in small batch size
def make_batch():
    batch = []
    positive = negative = 0 ## 为了记录NSP任务中的正样本和负样本的个数，比例最好是在一个batch中接近1：1
    # bacth_size/2=3；这里是为了保证一个batch中有3个正样本3两个负样本；这里的batch_size=6
    while positive != batch_size/2 or negative != batch_size/2:
        # randrange() 方法返回指定递增基数集合中的一个随机数，基数默认值为1。
        # 这里是为了从整个样本（目前只有6个）中随机抽取两个句子，一个作为tokens_a，一个作为tokens_b
        tokens_a_index, tokens_b_index= randrange(len(sentences)), randrange(len(sentences)) 
        # 比如tokens_a_index=3，tokens_b_index=1；从整个样本中抽取对应的样本；
        ## 根据索引获取对应样本：tokens_a=[5, 23, 26, 20, 9, 13, 18] tokens_b=[27, 11, 23, 8, 17, 28, 12, 22, 16, 25]
        tokens_a, tokens_b= token_list[tokens_a_index], token_list[tokens_b_index]
        # input_id 长度是 20
        ## 加上特殊符号，CLS符号是1，sep符号是2：[1, 5, 23, 26, 20, 9, 13, 18, 2, 27, 11, 23, 8, 17, 28, 12, 22, 16, 25, 2]
        input_ids = [word_dict['[CLS]']] + tokens_a + [word_dict['[SEP]']] + tokens_b + [word_dict['[SEP]']] 
        ##分割句子符号：[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
        # 这里需要注意的是，这里的segment_ids是为了区分两个句子的，所以第一个句子的segment_ids都是0，第二个句子的segment_ids都是1
        # 而 [sep]符号是第二个句子的,segment_ids是1,[CLS]符号是第一个句子的，segment_ids是0
        segment_ids = [0] * (1 + len(tokens_a) + 1) + [1] * (len(tokens_b) + 1)

        # MASK LM
        # 最大mask数量：max_pred=3；整个句子的15%的字符可以被mask掉，这里取和max_pred中的最小值
        n_pred =  min(max_pred, max(1, int(round(len(input_ids) * 0.15)))) # n_pred=3；整个句子的15%的字符可以被mask掉，这里取和max_pred中的最小值，确保每次计算损失的时候没有那么多字符以及信息充足，有15%做控制就够了；其实可以不用加这个，单个句子少了，就要加上足够的训练样本
        ## cand_maked_pos=[1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]；
        # 整个句子input_ids中可以被mask的符号必须是非cls和sep符号的，要不然没意义，导致模型无法学习到有用的信息；
        cand_maked_pos = [i for i, token in enumerate(input_ids)
                          if token != word_dict['[CLS]'] and token != word_dict['[SEP]']] 
        # 打乱顺序
        shuffle(cand_maked_pos)## 打乱顺序：cand_maked_pos=[6, 5, 17, 3, 1, 13, 16, 10, 12, 2, 9, 7, 11, 18, 4, 14, 15]  其实取mask对应的位置有很多方法，这里只是一种使用shuffle的方式
        masked_tokens, masked_pos = [], []
        ## 取其中的三个；masked_pos=[6, 5, 17] 注意这里对应的是position信息；masked_tokens=[13, 9, 16] 注意这里是被mask的元素之前对应的原始单字数字；
        # 选中 15%mask(这里有一个最小的限制1个，最大的限制是max_pred个)； 
        # 遵循 80%的概率mask，10%的概率替换，10%的概率不变
        for pos in cand_maked_pos[:n_pred]:
            # pos 是把input_ids中的cls和sep符号去掉之后的位置信息，所以这里的pos是从1开始的；
            # mask的pos和对应的原始单字数字
            masked_pos.append(pos)
            masked_tokens.append(input_ids[pos])
            if random() < 0.8:  # 80%
                # 替换为[MASK]对应的词表数字
                input_ids[pos] = word_dict['[MASK]'] # make mask
            elif random() < 0.5:  # 10% = 20% * 50%
                index = randint(0, vocab_size - 1) # random index in vocabulary
                input_ids[pos] = word_dict[number_dict[index]] # replace

        # Zero Paddings
        # 填充到最大长度；maxlen=30；因为 input_id长度是20，因此 n_pad=10
        n_pad = maxlen - len(input_ids)##maxlen=30；n_pad=10
        input_ids.extend([0] * n_pad)#在input_ids后面补零
        segment_ids.extend([0] * n_pad)# 在segment_ids 后面补零；这里有一个问题，0和之前的重了，这里主要是为了区分不同的句子，所以无所谓啊；他其实是另一种维度的位置信息；

        # Zero Padding (100% - 15%) 
        # tokens 是为了计算一个batch中句子的mlm损失的时候可以组成一个有效矩阵放进去；
        # 保证mask矩阵是一个有效的矩阵，也就是每个句子的mask数量是一样的；

        # 不然第一个句子预测5个字符，第二句子预测7个字符，第三个句子预测8个字符，组不成一个有效的矩阵；
        ## 这里非常重要，为什么是对masked_tokens是补零，而不是补其他的字符？？？？我补1可不可以？？
        # 补0 是因为 token为0,对应的是word_dict['[PAD]']，这个是不参与训练的，所以补0是最好的选择；
        # 第一次： max_pred=5；n_pred = 3
        # 矩阵一定是batch_size * max_pred的，所以这里要保证每个句子的mask数量是一样的；
        if max_pred > n_pred:
            n_pad = max_pred - n_pred
            # 补充0，在交叉熵损失函数中，ignore_index=0，也就是不参与计算损失的；
            masked_tokens.extend([0] * n_pad)##  masked_tokens= [13, 9, 16, 0, 0] masked_tokens 对应的是被mask的元素的原始真实标签是啥，也就是groundtruth
            # pos补充0 ？？
            # 但pos为0对应的一定是 [CLS]符号
            masked_pos.extend([0] * n_pad)## masked_pos= [6, 5, 17，0，0] masked_pos是记录哪些位置被mask了
        
        # positive和negative 初始化为0；这里是为了记录NSP任务中的正样本和负样本的个数，比例最好是在一个batch中接近1：1
        # 句子a,b相邻，为IsNext；同时positive<batch_size/2；
        # 句子a,b不相邻，为NotNext；同时negative<batch_size/2；
        # 这里的batch_size=6；所以一个batch中有3个正样本3两个负样本；
        if tokens_a_index + 1 == tokens_b_index and positive < batch_size/2:
            # IsNext ；这里的True是为了区分是IsNext还是NotNext
            # input_ids 原始句子的数字化表示，也经过了mask处理和padding处理
            # segment_ids 句子的分割符号，也经过了padding处理，主要区分两个句子
            # masked_tokens 被mask的元素之前对应的原始单字数字；也被padding处理了
            # masked_pos 被mask的位置,也被padding处理了
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, True]) # IsNext
            positive += 1
        elif tokens_a_index + 1 != tokens_b_index and negative < batch_size/2:
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, False]) # NotNext
            negative += 1
    return batch
# Proprecessing Finished

# 和transformer是一样的，这里的mask是为了在计算attention的时候，把padding的位置mask掉，不参与计算；
def get_attn_pad_mask(seq_q, seq_k):
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    # eq(zero) is PAD token
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1)  # batch_size x 1 x len_k(=len_q), one is masking
    return pad_attn_mask.expand(batch_size, len_q, len_k)  # batch_size x len_q x len_k

# 选中的代码定义了一个名为 gelu 的函数，该函数实现了高斯误差线性单元（Gaussian Error Linear Unit, GELU）激活函数。
# GELU 是一种非线性激活函数，常用于神经网络中，特别是在自然语言处理任务中。
# GELU 激活函数的定义如下：
# $$ \text{GELU}(x) = x \cdot \Phi(x) $$
# 其中，$\Phi(x)$ 是标准正态分布的累积分布函数（CDF），可以近似为：
# $$ \Phi(x) \approx 0.5 \cdot (1 + \text{erf}(\frac{x}{\sqrt{2}})) $$
# 这里，$\text{erf}(x)$ 是误差函数。
def gelu(x):
    "Implementation of the gelu activation function by Hugging Face"
    return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))

class Embedding(nn.Module):
    def __init__(self):
        super(Embedding, self).__init__()
        # 词表大小，词向量维度 [vocab_size, d_model] (29,768)
        self.tok_embed = nn.Embedding(vocab_size, d_model)  # token embedding
        # 位置编码的最大长度，词向量维度 [maxlen, d_model] (30,768)
        # 位置embedding 是一个可学习的参数，需要在训练中进行更新，这和transformer中的位置编码是不一样的；
        self.pos_embed = nn.Embedding(maxlen, d_model)  # position embedding
        # 分割符号的个数，词向量维度 [n_segments, d_model] (2,768)
        self.seg_embed = nn.Embedding(n_segments, d_model)  # segment(token type) embedding
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x, seg):
        # x: input_ids 原始句子的数字化表示，也经过了mask处理和padding处理 [batch_size, maxlen] (6,30)
        # seg: segment_ids 句子的分割符号，也经过了padding处理，主要区分两个句子 [batch_size, maxlen] (6,30)
        # seq_len: 句子的长度；这里的seq_len=30
        seq_len = x.size(1)
        pos = torch.arange(seq_len, dtype=torch.long)
        pos = pos.unsqueeze(0).expand_as(x)  # (seq_len,) -> (batch_size, seq_len)
        embedding = self.tok_embed(x) + self.pos_embed(pos) + self.seg_embed(seg)
        # 归一化 [batch_size, maxlen, d_model] (6,30,768)
        return self.norm(embedding)

class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, Q, K, V, attn_mask):
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k) # scores : [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
        scores.masked_fill_(attn_mask, -1e9) # Fills elements of self tensor with value where mask is one.
        attn = nn.Softmax(dim=-1)(scores)
        context = torch.matmul(attn, V)
        return context, attn

class MultiHeadAttention(nn.Module):
    def __init__(self):
        super(MultiHeadAttention, self).__init__()
        self.W_Q = nn.Linear(d_model, d_k * n_heads)
        self.W_K = nn.Linear(d_model, d_k * n_heads)
        self.W_V = nn.Linear(d_model, d_v * n_heads)
    def forward(self, Q, K, V, attn_mask):
        # q: [batch_size x len_q x d_model], k: [batch_size x len_k x d_model], v: [batch_size x len_k x d_model]
        residual, batch_size = Q, Q.size(0)
        # (B, S, D) -proj-> (B, S, D) -split-> (B, S, H, W) -trans-> (B, H, S, W)
        q_s = self.W_Q(Q).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # q_s: [batch_size x n_heads x len_q x d_k]
        k_s = self.W_K(K).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # k_s: [batch_size x n_heads x len_k x d_k]
        v_s = self.W_V(V).view(batch_size, -1, n_heads, d_v).transpose(1,2)  # v_s: [batch_size x n_heads x len_k x d_v]
        
        attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1) # attn_mask : [batch_size x n_heads x len_q x len_k]

        # context: [batch_size x n_heads x len_q x d_v], attn: [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
        context, attn = ScaledDotProductAttention()(q_s, k_s, v_s, attn_mask)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, n_heads * d_v) # context: [batch_size x len_q x n_heads * d_v]
        output = nn.Linear(n_heads * d_v, d_model)(context)
        return nn.LayerNorm(d_model)(output + residual), attn # output: [batch_size x len_q x d_model]

class PoswiseFeedForwardNet(nn.Module):
    def __init__(self):
        super(PoswiseFeedForwardNet, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        # (batch_size, len_seq, d_model) -> (batch_size, len_seq, d_ff) -> (batch_size, len_seq, d_model)
        return self.fc2(gelu(self.fc1(x)))

class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet()

    def forward(self, enc_inputs, enc_self_attn_mask):
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask) # enc_inputs to same Q,K,V
        enc_outputs = self.pos_ffn(enc_outputs) # enc_outputs: [batch_size x len_q x d_model]
        return enc_outputs, attn

## 1. BERT模型整体架构
class BERT(nn.Module):
    def __init__(self):
        super(BERT, self).__init__()
        # 词嵌入层
        self.embedding = Embedding() ## 词向量层，构建词表矩阵
        # 6个encoder堆叠起来
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)]) ## 把N个encoder堆叠起来，具体encoder实现一会看
        ## 前馈神经网络-cls
        self.fc = nn.Linear(d_model, d_model) 
        self.activ1 = nn.Tanh() ## 激活函数-cls
        #-mlm
        self.linear = nn.Linear(d_model, d_model)
        self.activ2 = gelu ## 激活函数--mlm
        # layer norm
        self.norm = nn.LayerNorm(d_model)
        ## cls 这是一个分类层，维度是从d_model到2，对应我们架构图中就是这种：[768,2]
        self.classifier = nn.Linear(d_model, 2)
        # decoder is shared with embedding layer
        # 共享权重，这里的decoder是和embedding层共享权重的，也就是说decoder的权重和embedding层的权重是一样的
        # self.tok_embed = nn.Embedding(vocab_size, d_model)  # token embedding

        # ? 还是有些不太明白
        embed_weight = self.embedding.tok_embed.weight
        n_vocab, n_dim = embed_weight.size()
        # 输入是d_model，输出是n_vocab，这里的decoder是一个线性层，也就是一个全连接层
        self.decoder = nn.Linear(n_dim, n_vocab, bias=False)
        self.decoder.weight = embed_weight
        self.decoder_bias = nn.Parameter(torch.zeros(n_vocab))


       

        
        

    def forward(self, input_ids, segment_ids, masked_pos):
        # input_ids 原始句子的数字化表示，也经过了mask处理和padding处理 [batch_size, maxlen] (6,30)
        # segment_ids 句子的分割符号，也经过了padding处理，主要区分两个句子 [batch_size, maxlen] (6,30)
        # masked_tokens 被mask的元素之前对应的原始单字数字；也被padding处理了 [batch_size, max_pred] (6,5)
        # masked_pos 被mask的位置,也被padding处理了 [batch_size, max_pred] (6,5)

        # output : [batch_size, maxlen, d_model] (6,30,768)
        output = self.embedding(input_ids, segment_ids)## 生成input_ids对应的embdding；和segment_ids对应的embedding
        # 和transformer是一样的，这里的mask是为了在计算attention的时候，把padding的位置mask掉，不参与计算；
        # ecn_self_attn_mask : [batch_size, maxlen, maxlen] (6,30,30)
        enc_self_attn_mask = get_attn_pad_mask(input_ids, input_ids)
        for layer in self.layers:
            # 输入
            # output : [batch_size, maxlen, d_model] (6,30,768)
            # enc_self_attn_mask : [batch_size, n_heads, maxlen, maxlen] (6,8,30,30)

            # 输出
            # output : [batch_size, maxlen, d_model] (6,30,768)
            # enc_self_attn : [batch_size, n_heads, maxlen, maxlen] (6,8,30,30)
            output, enc_self_attn = layer(output, enc_self_attn_mask)
        # output : [batch_size, len, d_model], attn : [batch_size, n_heads, d_mode, d_model]

        # it will be decided by first token(CLS)
        # output[:, 0]) : [batch_size, d_model] (6,768)
        # 原因是取了batch中的第一个句子的第一个位置的embedding，这个位置是cls符号的位置，也就是说cls符号的embedding是整个句子的embedding；

        # h_pooled : [batch_size, d_model] (6,768)
        h_pooled = self.activ1(self.fc(output[:, 0])) # [batch_size, d_model]
        # cls 分类层
        logits_clsf = self.classifier(h_pooled) # [batch_size, 2]

        
        # [batch_size, max_pred, d_model]  其中一个 masked_pos= [6, 5, 17，0，0]
        # 这里扩充是把每一个元素复制到d_model维度上，也就是说一个元素变成了d_model个元素
        masked_pos = masked_pos[:, :, None].expand(-1, -1, output.size(-1)) 
        # print('output\n',output.data)
        # print('masked_pos\n',masked_pos)
        # get masked position from final output of transformer.
        # masking position [batch_size, max_pred, d_model] (6,5,768)
        h_masked = torch.gather(output, 1, masked_pos) 
        # print('h_masked\n',h_masked)
        # h_masked [batch_size, max_pred, d_model] (6,5,768)
        h_masked = self.norm(self.activ2(self.linear(h_masked)))
        # logits_lm [batch_size, max_pred, n_vocab] (6,5,29) 
        logits_lm = self.decoder(h_masked) + self.decoder_bias 
        
        # logits_clsf [batch_size, 2] (6,2)
        # logits_lm [batch_size, max_pred, n_vocab] (6,5,29)

        return logits_lm, logits_clsf
#---------------------------------------------------#
#   设置种子
#---------------------------------------------------#
def seed_everything(myseed=11):
    seed(myseed)
    np.random.seed(myseed)
    torch.manual_seed(myseed)
    torch.cuda.manual_seed(myseed)
    torch.cuda.manual_seed_all(myseed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


前 masked_pos: tensor([[ 1,  4, 10,  0,  0],
[ 1,  0,  0,  0,  0],
[ 9,  4,  0,  0,  0],
[ 1,  0,  0,  0,  0],
[ 7,  0,  0,  0,  0],
[10,  2,  0,  0,  0]])


后 masked_pos: tensor([[[ 1,  1,  1,  ...,  1,  1,  1],
        [ 4,  4,  4,  ...,  4,  4,  4],
        [10, 10, 10,  ..., 10, 10, 10],
        [ 0,  0,  0,  ...,  0,  0,  0],
        [ 0,  0,  0,  ...,  0,  0,  0]],

[[ 1,  1,  1,  ...,  1,  1,  1],
        [ 0,  0,  0,  ...,  0,  0,  0],
        [ 0,  0,  0,  ...,  0,  0,  0],
        [ 0,  0,  0,  ...,  0,  0,  0],
        [ 0,  0,  0,  ...,  0,  0,  0]],

[[ 9,  9,  9,  ...,  9,  9,  9],
        [ 4,  4,  4,  ...,  4,  4,  4],
        [ 0,  0,  0,  ...,  0,  0,  0],
        [ 0,  0,  0,  ...,  0,  0,  0],
        [ 0,  0,  0,  ...,  0,  0,  0]],

[[ 1,  1,  1,  ...,  1,  1,  1],
        [ 0,  0,  0,  ...,  0,  0,  0],
        [ 0,  0,  0,  ...,  0,  0,  0],
        [ 0,  0,  0,  ...,  0,  0,  0],
        [ 0,  0,  0,  ...,  0,  0,  0]],

[[ 7,  7,  7,  ...,  7,  7,  7],
        [ 0,  0,  0,  ...,  0,  0,  0],
        [ 0,  0,  0,  ...,  0,  0,  0],
        [ 0,  0,  0,  ...,  0,  0,  0],
        [ 0,  0,  0,  ...,  0,  0,  0]],

[[10, 10, 10,  ..., 10, 10, 10],
        [ 2,  2,  2,  ...,  2,  2,  2],
        [ 0,  0,  0,  ...,  0,  0,  0],
        [ 0,  0,  0,  ...,  0,  0,  0],
        [ 0,  0,  0,  ...,  0,  0,  0]]])

## 主要运行部分：

In [4]:
# BERT Parameters
maxlen = 30 # 句子的最大长度 cover住95% 不要看平均数 或者99%  直接取最大可以吗？当然也可以，看你自己
batch_size = 6 # 每一组有多少个句子一起送进去模型
max_pred = 5  # max tokens of prediction
n_layers = 6 # number of Encoder of Encoder Layer
n_heads = 12 # number of heads in Multi-Head Attention
d_model = 768 # Embedding Size
d_ff = 3072  # 4*d_model, FeedForward dimension
d_k = d_v = 64  # dimension of K(=Q), V
n_segments = 2

text = (
    'Hello, how are you? I am Romeo.\n'
    'Hello, Romeo My name is Juliet. Nice to meet you.\n'
    'Nice meet you too. How are you today?\n'
    'Great. My baseball team won the competition.\n'
    'Oh Congratulations, Juliet\n'
    'Thanks you Romeo'
)
sentences = re.sub("[.,!?\\-]", '', text.lower()).split('\n')  # filter '.', ',', '?', '!'
word_list = list(set(" ".join(sentences).split()))
word_dict = {'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3}
# 词表的构建，因为是bert，所以需要加上这几个特殊符号，因此词表的长度是len(word_list)+4
for i, w in enumerate(word_list):
    word_dict[w] = i + 4
# 词表的反转，方便根据索引获取对应的词
number_dict = {i: w for i, w in enumerate(word_dict)}
# {0: '[PAD]', 1: '[CLS]', 2: '[SEP]', 3: '[MASK]', 4: 'thanks', 5: 'team', 6: 'great', 7: 'juliet', 8: 'name', 9: 'how', 10: 'the', 11: 'my', 12: 'nice', 13: 'too', 14: 'are', 15: 'to', 16: 'competition', 17: 'baseball', 18: 'romeo', 19: 'won', 20: 'i', 21: 'is', 22: 'meet', 23: 'congratulations', 24: 'oh', 25: 'hello', 26: 'today', 27: 'you', 28: 'am'}

# 词表的长度 29
vocab_size = len(word_dict)
#  把句子转换成数字序列
token_list = list()
for sentence in sentences:
    arr = [word_dict[s] for s in sentence.split()]
    token_list.append(arr)
# print(token_list)

batch = make_batch()

# map 函数会将指定的函数（这里是 torch.LongTensor）应用到 zip(*batch) 生成的每个元组上。
# torch.LongTensor 是 PyTorch 中的一种张量类型，表示 64 位整数类型的张量。
# 因此，这段代码的作用是将 zip(*batch) 生成的每个元组转换为 LongTensor 类型。
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(*batch))
# print('input_ids:\n', input_ids,'segment_ids:\n', segment_ids,'masked_tokens:\n', masked_tokens,'masked_pos:\n', masked_pos,'isNext:\n', isNext)
# input_ids 原始句子的数字化表示，也经过了mask处理和padding处理 [batch_size, maxlen] (6,30)
# segment_ids 句子的分割符号，也经过了padding处理，主要区分两个句子 [batch_size, maxlen] (6,30)
# masked_tokens 被mask的元素之前对应的原始单字数字；也被padding处理了 [batch_size, max_pred] (6,5)
# masked_pos 被mask的位置,也被padding处理了 [batch_size, max_pred] (6,5)
# isNext 用于判断两个句子是否相邻 [batch_size] (6)


model = BERT()
# 交叉熵损失函数，softmax+log+NLLLoss
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(model.parameters(), lr=0.001)



In [5]:
# for epoch in range(100):
for epoch in range(100):
    optimizer.zero_grad()
    # logits_clsf [batch_size, 2] (6,2)
    # MLM 掩码语言模型，预测被mask的字符
    # logits_lm [batch_size, max_pred, n_vocab] (6,5,29)
    logits_lm, logits_clsf = model(input_ids, segment_ids, masked_pos)## logits_lm 【6，5，29】 bs*max_pred*voca  logits_clsf:[6*2]
    # for masked LM ;masked_tokens [6,5]
    #  计算预测的masked的字符和真实的masked字符之间的交叉熵损失

    # 交叉熵损失函数的输入需要满足：
    # 预测值（logits）：形状为 [batch_size, n_vocab, *dims]，其中 n_vocab 是分类的类别数。
    # 真实标签（targets）：形状为 [batch_size, *dims]，每个位置是类别的索引（整数，范围 [0, n_vocab-1]）。
    # 二者的维度关系是：targets 的维度比 logits 少一个 n_vocab 维度。
    # logits_lm.transpose(1, 2).size: torch.Size([6, 29, 5]) [batch_size, n_vocab, max_pred]
    # masked_tokens.size: torch.Size([6, 5]) [batch_size, max_pred]

    # 在Transformer等模型中，通常输出的 logits 形状是 [batch_size, seq_len, n_vocab]
    # （即 [6,5,29]）。但 nn.CrossEntropyLoss 要求 n_vocab 维度位于第二个位置（[batch_size, n_vocab, seq_len]）
    # ，因此需要通过 transpose(1, 2) 调整维度顺序。
    loss_lm = criterion(logits_lm.transpose(1, 2), masked_tokens) 

    #  平均损失
    loss_lm = (loss_lm.float()).mean()
    # nn.CrossEntropyLoss 默认返回的是批次内所有样本损失的平均值
    # loss_lm = loss_lm.float()
    loss_clsf = criterion(logits_clsf, isNext) # for sentence classification
    loss = loss_lm + loss_clsf
    if (epoch + 1) % 10 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
    loss.backward()
    optimizer.step()

Epoch: 0010 cost = 145.255295
Epoch: 0020 cost = 46.237587
Epoch: 0030 cost = 20.768850
Epoch: 0040 cost = 14.773284
Epoch: 0050 cost = 9.951262
Epoch: 0060 cost = 7.177818
Epoch: 0070 cost = 5.248575
Epoch: 0080 cost = 4.148743
Epoch: 0090 cost = 2.761411
Epoch: 0100 cost = 2.976523


In [6]:
# batch.size() (6,5)
# batch[0]: [[1, 6, 21, 17, 3, 24, 11, 25, 22, 3, 13, 2, 6, 10, 5, 13, 8, 3, 21, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
# [15, 14, 7, 0, 0], [9, 17, 4, 0, 0], False]
# print('batch.size():',np.array(batch).shape,'batch:',batch,'batch[0]:',batch[0])
# Predict mask tokens ans isNext
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(batch[0]))
print('--'*20)
print('text:')
print(text)
print('--'*20)
print('input_ids对应的句子(除去[PAD]符号)：')
print([number_dict[w.item()] for w in input_ids[0] if number_dict[w.item()] != '[PAD]'])
print('--'*20)
# Predict mask tokens
# MLM 掩码语言模型，预测被mask的字符
# logits_lm [batch_size, max_pred, n_vocab] (1,5,29)
# logits_clsf [batch_size, 2] (1,2)
logits_lm, logits_clsf = model(input_ids, segment_ids, masked_pos)


# data.max(2) 方法沿着第三个维度（即词汇表维度）找到最大值。max(2) 返回两个张量：
# 最大值和最大值的索引。通过 [1] 选择最大值的索引，然后通过 [0] 选择第一个批次的结果。
# 最后，使用 data.numpy() 将结果转换为 NumPy 数组。
# logits_lm.data [1,5,29]
print('logits_lm.data',logits_lm.data.shape)
# max(2,keepdim=True) 保持维度不变，取最大值索引 max(2)[1]
# logits_lm.data.max(2)[1] 取出来的是最大值的索引列 indices 维度是[1,5]，
# logits_lm.data.max(2)[1][0] 是indices 的第一行，维度是[5]，也就是预测的mask的字符的索引
# logits_lm.data.max(2)[1][0]依然是tensor([22, 16, 22, 13, 13]),
# 因此.data.numpy() 先取出数据，然后转换为numpy数组.

# logits_lm.data.max(2): torch.return_types.max(
# values=tensor([[34.8164, 34.9115, 34.8043, 41.5092, 41.5092]]),
# indices=tensor([[22, 16, 22, 13, 13]]))
# print('logits_lm.data.max(2):',logits_lm.data.max(2))
logits_lm = logits_lm.data.max(2)[1][0].data.numpy()
print('masked tokens list : ',[pos.item() for pos in masked_tokens[0] if pos.item() != 0])
# 根据maskd_pos不为0的位置，取出来对应的预测的mask的字符的索引
print('predict masked tokens list : ',[pos for i,pos in enumerate(logits_lm) if masked_pos[0][i] != 0])

# logits_clsf [batch_size, 2] (1,2)
# max(1)坍缩第二个维度，[1]取最大值索引,data取出数据，numpy转换为numpy数组,只有一个元素,所以取[0]
# logits_clsf = logits_clsf.data.max(1)[1].data.numpy()[0]
# 上面的不对,取出来索引
# logits_clsf = logits_clsf.data.max(1)[1]

# logits_clsf: tensor([[-4.7248,  5.4548]], grad_fn=<AddmmBackward0>)
print('logits_clsf:',logits_clsf)

# loss_clsf: tensor(2.0702e-05, grad_fn=<NllLossBackward0>)
print('loss_clsf:',loss_clsf)
print('isNext : ', True if isNext else False)
# 假设 logits_clsf 是一个包含两个类的 logits,使用 argmax() 函数找到最大值的索引。
predicted_class = torch.argmax(logits_clsf).item()
print('predict isNext : ', True if predicted_class == 1 else False)

----------------------------------------
text:
Hello, how are you? I am Romeo.
Hello, Romeo My name is Juliet. Nice to meet you.
Nice meet you too. How are you today?
Great. My baseball team won the competition.
Oh Congratulations, Juliet
Thanks you Romeo
----------------------------------------
input_ids对应的句子(除去[PAD]符号)：
['[CLS]', 'hello', 'how', '[MASK]', 'you', 'i', 'am', 'name', '[SEP]', '[MASK]', 'meet', 'you', 'too', 'how', 'are', 'you', 'today', '[SEP]']
----------------------------------------
logits_lm.data torch.Size([1, 5, 29])
masked tokens list :  [24, 7, 22]
predict masked tokens list :  [np.int64(22), np.int64(22), np.int64(22)]
logits_clsf: tensor([[-10.4683,  10.7951]], grad_fn=<AddmmBackward0>)
loss_clsf: tensor(0., grad_fn=<NllLossBackward0>)
isNext :  False
predict isNext :  True
